In [3]:
import numpy as np

# Базовая задача МНК

# Централизация признаков и МНК.

1. Когда мы центрируем признаки в X, среднее значение каждого признака становится нулевым. Аналогично, как мы работаем с отклонениями от среднего, что исключает необходимость учитывать свободный член в модели. (X^T X)^-1 для центрированных данных будет соответствовать только влиянию признаков без необходимости учитывать свободный член.

2. Добавление столбца единиц в матрицу X эквивалентно добавлению свободного члена в модель, что позволяет модели учитывать свободный член. При вычислении (X^T X)^-1 для расширенной матрицы, последний столбец и строка представляют влияние свободного члена. Удаление этой строки и столбца возвращает к случаю, когда мы не учитываем свободный член —  ситуация становится аналогичной центрированным признакам.

In [5]:
np.random.seed(42)
X = np.random.randn(10, 5)

X1 = X - X.mean(axis=0) 
X1_final = np.linalg.inv(X1.T @ X1) 

X2 = np.hstack([X, np.ones((X.shape[0], 1))]) 
X2_1 = np.linalg.inv(X2.T @ X2) 
X2_final = X2_1[:-1, :-1]

are_equal = np.allclose(X1_final, X2_final)

are_equal

True